# Miniproject 3

In [ ]:
!pip install pyperplan


     |████████████████████████████████| 69 kB 4.4 MB/s 


## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
from pyperplan.pddl.parser import Parser
from pyperplan import grounding, planner
import numpy as np
import os
import tempfile


BLOCKS_DOMAIN = """(define (domain blocks)
    (:requirements :strips :typing)
    (:types block)
    (:predicates 
        (on ?x - block ?y - block)
        (ontable ?x - block)
        (clear ?x - block)
        (handempty)
        (handfull)
        (holding ?x - block)
    )

    (:action pick-up
        :parameters (?x - block)
        :precondition (and
            (clear ?x) 
            (ontable ?x) 
            (handempty)
        )
        :effect (and
            (not (ontable ?x))
            (not (clear ?x))
            (not (handempty))
            (handfull)
            (holding ?x)
        )
    )

    (:action put-down
        :parameters (?x - block)
        :precondition (and 
            (holding ?x)
            (handfull)
        )
        :effect (and 
            (not (holding ?x))
            (clear ?x)
            (handempty)
            (not (handfull))
            (ontable ?x))
        )

    (:action stack
        :parameters (?x - block ?y - block)
        :precondition (and
            (holding ?x) 
            (clear ?y)
            (handfull)
        )
        :effect (and 
            (not (holding ?x))
            (not (clear ?y))
            (clear ?x)
            (handempty)
            (not (handfull))
            (on ?x ?y)
        )
    )

    (:action unstack
        :parameters (?x - block ?y - block)
        :precondition (and
            (on ?x ?y)
            (clear ?x)
            (handempty)
        )
        :effect (and 
            (holding ?x)
            (clear ?y)
            (not (clear ?x))
            (not (handempty))
            (handfull)
            (not (on ?x ?y))
        )
    )
)
"""

BLOCKS_PROBLEM = """(define (problem blocks)
    (:domain blocks)
    (:objects 
        d - block
        b - block
        a - block
        c - block
    )
    (:init 
        (clear a) 
        (on a b) 
        (on b c)
        (on c d)
        (ontable d) 
        (handempty)
    )
    (:goal (and (on d c) (on c b) (on b a)))
)
"""


class SearchAndRescueSimulator:
  """A simulator for a search and rescue problem.

  In search and rescue, a robot must navigate to, pick up, and
  drop off people that are in need of help.

  States are dictionaries with the following attributes:
    "obstacle_map": A numpy array of 0s and 1s, where a 0
      represents free space and a 1 represents an obstacle.
    "robot": A (row, col) representing the robot's loc.
    "hospital_loc": A (row, col) representing the hospital's loc.
    "carrying": The str name of a person being carried,
      or None, if no person is being carried.
    "people": A dict mapping str people names to (row, col)
      locs. If a person is being carried, they do not
      appear in this dict.

  Actions are strs. The following actions are defined:
    "up" / "down" / "left" / "right" : Moves the robot. The
      robot cannot move into obstacles or off the map.
    "pickup-{person}": If the robot is at the person, and if
      the robot is not already carrying someone, picks.
    "dropoff": If the robot is carrying a person, they are
      dropped off at the robot's current location.

  There is one constant initial state. See `get_initial_state`
  and `render_state`.

  Example usage:
    simulator = SearchAndRescueSimulator()
    state = simulator.get_initial_state()
    simulator.pretty_print_state(state)
    action = "down"
    next_state = simulator.get_next_state(state, action)
  """
  def get_initial_state(self):
    obstacle_map = np.array([
      [0, 0, 0, 0, 0, 0, 0],
      [0, 1, 1, 0, 0, 1, 1],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 1, 0, 0, 0, 0],
      [0, 0, 1, 0, 1, 0, 0],
      [0, 0, 0, 0, 0, 1, 0],
      [0, 1, 0, 0, 1, 0, 0]
    ], dtype=np.uint8)

    robot = (0, 0)  # top left corner
    hospital = (6, 6)  # bottom right corner
    carrying = None
    people = {
      "p1": (4, 0),
      "p2": (6, 0),
      "p3": (0, 6),
      "p4": (3, 3)
    }

    return dict(
      obstacle_map=obstacle_map,
      robot=robot,
      hospital_loc=hospital,
      carrying=carrying,
      people=people
    )

  def pretty_print_state(self, state):
    height, width = state["obstacle_map"].shape
    state_arr = np.full((height, width), "  ", dtype=object)
    state_arr[state["obstacle_map"] == 1] = "##"
    state_arr[state["hospital_loc"]] = "Ho"
    state_arr[state["robot"]] = "Ro"
    for person, loc in state["people"].items():
      if loc == state["hospital_loc"]:
        continue
      elif loc == state["robot"]:
        person = "R" + person[-1]
      elif loc == state["hospital_loc"]:
        continue
      state_arr[loc] = person
    # Add padding
    padded_state_arr = np.full((height+2, width+2), "##", dtype=object)
    padded_state_arr[1:-1, 1:-1] = state_arr
    state_arr = padded_state_arr
    carrying_str = f"Carrying: {state['carrying']}"
    for row in state_arr:
      print(''.join(row))
    print(carrying_str)
    print()

  def get_next_state(self, state, action):
    legal_actions = ["up", "down", "left", "right", "dropoff"]
    for person in state["people"]:
      legal_actions.append(f"pickup-{person}")
    if action not in legal_actions:
      raise ValueError(f"Unrecognized action {action}. Actions must be one of: {legal_actions}")

    if action in ["up", "down", "left", "right"]:
      dr, dc = {
        "up": (-1, 0),
        "down": (1, 0),
        "left": (0, -1),
        "right": (0, 1),
      }[action]

      r, c = state["robot"]

      if not (0 <= r + dr < state["obstacle_map"].shape[0] and \
              0 <= c + dc < state["obstacle_map"].shape[1]):
        print("WARNING: attempted to move out of bounds, action has no effect.")
        return state

      if state["obstacle_map"][r+dr, c+dc]:
        print("WARNING: attempted to move into an obstacle, action has no effect.")
        return state

      new_state = self._copy_state(state)
      new_state["robot"] = (r + dr, c + dc)

      return new_state

    elif action.startswith("pickup"):
      person = action.split("-")[1]

      if state["carrying"] is not None:
        print("WARNING: attempted to pick up a person while already carrying someone, action has no effect.")
        return state        

      if person not in state["people"] or (state["people"][person] != state["robot"]):
        print("WARNING: attempted to pick up a person not at the robot, action has no effect.")
        return state

      new_state = self._copy_state(state)
      del new_state["people"][person]
      new_state["carrying"] = person

      return new_state

    assert action == "dropoff"
    if state["carrying"] is None:
      print("WARNING: attempted to dropoff while not carrying anyone, action has no effect.")
      return state

    person = state["carrying"]
    new_state = self._copy_state(state)
    new_state["carrying"] = None
    new_state["people"][person] = state["robot"]

    return new_state   

  def _copy_state(self, state):
    return dict(
      obstacle_map=state["obstacle_map"],  # static
      robot=state["robot"],
      hospital_loc=state["hospital_loc"],
      carrying=state["carrying"],
      people=state["people"].copy()
    )


def execute_sar_plan(plan, verbose=False):
  """Execute a plan for search and rescue.

  Args:
    plan: A list of action strs, see SearchAndRescueSimulator.
    verbose: If true, print all the states.

  Returns:
    final_state: A SearchAndRescueSimulator state.
  """
  simulator = SearchAndRescueSimulator()
  state = simulator.get_initial_state()
  if verbose:
    simulator.pretty_print_state(state)
  for action in plan:
    state = simulator.get_next_state(state, action)
    if verbose:
      print("Executed action:", action)
      simulator.pretty_print_state(state)
  return state


def count_num_delivered(plan, verbose=True):
  """Execute a plan for search and rescue and count the number of
    people delivered.

  Args:
    plan: A list of action strs, see SearchAndRescueSimulator.
    verbose: If true, print all the states.

  Returns:
    num_delivered: int
  """
  state = execute_sar_plan(plan, verbose=verbose)
  num_delivered = 0
  for loc in state["people"].values():
    if loc == state["hospital_loc"]:
      num_delivered += 1
  return num_delivered


def check_sar_plan(plan, verbose=True):
  """Execute a plan for search and rescue and check the goal.

  Args:
    plan: A list of action strs, see SearchAndRescueSimulator.
    verbose: If true, print all the states.

  Returns:
    succeeded: bool
  """
  return count_num_delivered(plan, verbose=verbose) == 4


def run_planning(domain_pddl_str, problem_pddl_str, search_alg_name,
                 heuristic_name=None):
  """Plan a sequence of actions to solve the given PDDL problem.

  This function is a lightweight wrapper around pyperplan.

  Args:
    domain_pddl_str: A str, the contents of a domain.pddl file.
    problem_pddl_str: A str, the contents of a problem.pddl file.
    search_alg_name: A str, the name of a search algorithm in
      pyperplan. Options: astar, wastar, gbf, bfs, ehs, ids, sat.
    heuristic_name: A str, the name of a heuristic in pyperplan.
      Options: blind, hadd, hmax, hsa, hff, lmcut, landmark.

  Returns:
    plan: A list of actions; each action is a pyperplan Operator.
  """
  # Parsing the PDDDL
  domain_file = tempfile.NamedTemporaryFile(delete=False)
  problem_file = tempfile.NamedTemporaryFile(delete=False)
  with open(domain_file.name, 'w') as f:
    f.write(domain_pddl_str)
  with open(problem_file.name, 'w') as f:
    f.write(problem_pddl_str)
  parser = Parser(domain_file.name, problem_file.name)
  domain = parser.parse_domain()
  problem = parser.parse_problem(domain)
  os.remove(domain_file.name)
  os.remove(problem_file.name)

  # Ground the PDDL
  task = grounding.ground(problem)

  # Get the search alg
  search_alg = planner.SEARCHES[search_alg_name]

  if heuristic_name is None:
    return search_alg(task)

  # Get the heuristic
  heuristic = planner.HEURISTICS[heuristic_name](task)

  # Run planning
  return search_alg(task, heuristic)



## Problems

### Let's Make a Plan
Use `run_planning` to find a plan for the blocks problem defined at the top of the colab file (`BLOCKS_DOMAIN`, `BLOCKS_PROBLEM`).

  The `run_planning` function takes in a PDDL domain string, a PDDL problem string, the name of a search algorithm, and the name of a heuristic (if the search algorithm is informed). It then uses the Python planning library `pyperplan` to find a plan.

  The plan returned by `run_planning` is a list of pyperplan Operators. You should not need to manipulate these data structures directly in this homework, but if you are curious about the definition, see [here](https://github.com/aibasel/pyperplan/blob/master/pyperplan/task.py#L23).

  The search algs available in pyperplan are: `astar, wastar, gbf, bfs, ehs, ids, sat`. The heuristics available in pyperplan are: `blind, hadd, hmax, hsa, hff, lmcut, landmark`.

  For this question, use the `astar` search algorithm with the `lmcut` heuristic.

For reference, our solution is **1** lines of code.

In [ ]:
def planning_warmup():
  """Use run_planning to find a plan for the blocks problem
    defined at the top of the colab file (BLOCKS_DOMAIN, BLOCKS_PROBLEM).

    Use the astar search algorithm with the lmcut heuristic.

  Returns:
    plan: A list of actions; each action is a pyperplan Operator.
  """
  return run_planning(BLOCKS_DOMAIN,BLOCKS_PROBLEM,"astar","lmcut")

Tests

In [ ]:
def warmup_test1():
  plan = planning_warmup()
  assert len(plan) == 8
  assert plan[0].name == '(unstack a b)'

warmup_test1()
print('Tests passed.')

Tests passed.


### Fill in the Blanks
You've received PDDL domain and problem strings from your boss and you need to make a plan, pronto! Unfortunately, some of the PDDL is missing.

  Here's what you know. What you're trying to model is a newspaper delivery robot. The robot starts out at a "home base" where there are papers that it can pick up. The robot can hold arbitrarily many papers at once. It can then move around to different locations and deliver papers.

  Not all locations want a paper -- the goal is to satisfy all the locations that do want a paper.

  You also know:
  * There are 6 locations in addition to 1 for the homebase. Locations 1, 2, 3, and 4 want paper; locations 5 and 6 do not.
  * There are 8 papers at the homebase.
  * The robot is initially at the homebase with no papers packed.

  Use this description to complete the PDDL domain and problem.

  If you are running into issues debugging the PDDL, look ahead to the next question, where we describe a useful online PDDL editor, and some common PDDL writing pitfalls.
  

For reference, our solution is **88** lines of code.

In [ ]:
def pddl_warmup():
  """Creates a PDDL domain and problem strs for newspaper delivery.

  Returns:
    domain: str
    problem: str
  """
  domain_str = """(define (domain newspapers)
    (:requirements :strips :typing)
    (:types loc paper)
    (:predicates 
      (isHomeBase ?loc - loc)
      (at ?loc - loc)
      (carrying ?paper - paper)
      (unpacked ?paper - paper)
      (satisfied ?loc - loc)
    )
    
    (:action pick-up
      :parameters (?paper - paper ?loc - loc)
      :precondition (and
        (at ?loc)
        (isHomeBase ?loc)
        (unpacked ?paper)
      )
      :effect (and
        (not (unpacked ?paper))
        (carrying ?paper)
      )
    )
    
    (:action move
      :parameters (?from - loc ?to - loc)
      :precondition (and
        (at ?from) 
      )
      :effect (and
        (not (at ?from))
        (at ?to)
      )
    )
    
    (:action deliver
      :parameters (?paper - paper ?loc - loc)
      :precondition (and
        (at ?loc)
        (carrying ?paper)
      )
      :effect (and
        (not (carrying ?paper))
        (satisfied ?loc)
      )
    )
    
)"""
  
  problem_str = """(define (problem newspapers1) (:domain newspapers)
  (:objects
    loc-0 - loc
    loc-1 - loc
    loc-2 - loc
    loc-3 - loc
    loc-4 - loc
    loc-5 - loc
    loc-6 - loc
    paper-0 - paper
    paper-1 - paper
    paper-2 - paper
    paper-3 - paper
    paper-4 - paper
    paper-5 - paper
    paper-6 - paper
    paper-7 - paper
  )
  (:init 
    (at loc-0)
    (isHomeBase loc-0)
    (unpacked paper-0)
    (unpacked paper-1)
    (unpacked paper-2)
    (unpacked paper-3)
    (unpacked paper-4)
    (unpacked paper-5)
    (unpacked paper-6)
    (unpacked paper-7)
  )
  (:goal (and
    (satisfied loc-1)
    (satisfied loc-2)
    (satisfied loc-3)
    (satisfied loc-4)
  ))
)"""

  return domain_str, problem_str

Tests

In [ ]:
def warmup_test2():
  domain, problem = pddl_warmup()
  plan = run_planning(domain, problem, "gbf", "hadd")
  assert plan, "Failed to find a plan."
  picked_up_papers = set()
  satisfied_locs = set()
  for op in plan:
    if "pickup" in op.name:
      _, _, paper, _ = op.name.split(" ")
      assert paper not in picked_up_papers, \
        "Should not pick up the same paper twice"
      picked_up_papers.add(paper)
    elif "deliver" in op.name:
      _, loc = op.name.rsplit(" ", 1)
      assert loc.endswith(")")
      loc = loc[:-1]
      assert loc not in satisfied_locs, \
        "Should not deliver to the same place twice"
      satisfied_locs.add(loc)
  assert satisfied_locs == {"loc-1", "loc-2", "loc-3", "loc-4"}

warmup_test2()
print('Tests passed.')

Tests passed.


### Search and Rescue Warmup 1
Find the initial robot location in a SearchAndRescueSimulator.

For reference, our solution is **2** lines of code.

In [ ]:
def sar_warmup1(simulator):
  """Find the initial robot location in the SearchAndRescueSimulator.

  Args:
    simulator: A SearchAndRescueSimulator.

  Returns:
    robot_loc: A tuple of ints (row, col) representing the robot state.
  """
  return simulator.get_initial_state()["robot"]

Tests

In [ ]:
def sar_warmup_test1():
  simulator = SearchAndRescueSimulator()
  robot_loc = sar_warmup1(simulator)
  assert robot_loc == (0, 0)

sar_warmup_test1()
print('Tests passed.')

Tests passed.


### Search and Rescue Warmup 2
Check if a row and col have an obstacle in a SearchAndRescueSimulator state.

For reference, our solution is **1** lines of code.

In [ ]:
def sar_warmup2(sar_state, row, col):
  """Check if a row and col have an obstacle in a SearchAndRescueSimulator state.

  Args:
    sar_state: A SearchAndRescueSimulator state.
    row: An int.
    col: An int.

  Returns:
    has_obstacle: True if (row, col) has an obstacle in sar_state.
  """
  return sar_state["obstacle_map"][row][col]

Tests

In [ ]:
def sar_warmup_test2():
  simulator = SearchAndRescueSimulator()
  state = simulator.get_initial_state()
  assert sar_warmup2(state, 0, 0)  == False
  assert sar_warmup2(state, 0, 1)  == False
  assert sar_warmup2(state, 1, 1)  == True
  assert sar_warmup2(state, 1, 2)  == True

sar_warmup_test2()
print('Tests passed.')

Tests passed.


### Search and Rescue Warmup 3
Hand-code a list of actions that will deliver person 'p1' to the hospital location.

For reference, our solution is **1** lines of code.

In [ ]:
def sar_warmup3():
  """Hand-code a list of actions that will deliver person 'p1' to the hospital location.

  Returns:
    actions: A list of str actions that will take person p1 to the hospital loccation.
  """
  return ["down","down","down","down","pickup-p1","up","up","right","right","right","right","right","right","down","down","down","down","dropoff"]

Tests

In [ ]:
def sar_warmup_test3():
  assert count_num_delivered(sar_warmup3()) == 1

sar_warmup_test3()
print('Tests passed.')

##################
##Ro          p3##
##  ####    ######
##              ##
##    ##p4      ##
##p1  ##  ##    ##
##          ##  ##
##p2##    ##  Ho##
##################
Carrying: None

Executed action: down
##################
##            p3##
##Ro####    ######
##              ##
##    ##p4      ##
##p1  ##  ##    ##
##          ##  ##
##p2##    ##  Ho##
##################
Carrying: None

Executed action: down
##################
##            p3##
##  ####    ######
##Ro            ##
##    ##p4      ##
##p1  ##  ##    ##
##          ##  ##
##p2##    ##  Ho##
##################
Carrying: None

Executed action: down
##################
##            p3##
##  ####    ######
##              ##
##Ro  ##p4      ##
##p1  ##  ##    ##
##          ##  ##
##p2##    ##  Ho##
##################
Carrying: None

Executed action: down
##################
##            p3##
##  ####    ######
##              ##
##    ##p4      ##
##R1  ##  ##    ##
##          ##  ##
##p2##    ##  Ho##
############

### Search and Rescue
Make a plan to solve the search and rescue problem in SearchAndRescueSimulator.

When the output of this function is given to `check_sar_plan`, it should return True.

This function should do the following:

1. Create PDDL domain and problem strings for search and rescue.  The operators should work for any grid size, obstacles, people locations, and hospital location.
2. Invoke `run_planning` with the `gbf` search algorithm with the `hadd` heuristic.
3. Convert the output of run_planning (pyperplan Operators) into actions that can be given to the SearchAndRescueSimulator.

For reference, this function takes ~1-2 seconds to run with our implementation.
To get credit on catsoop, make sure that your function finishes in <10 seconds.

If you're not sure how on how to get started, scroll down to the bottom of the catsoop page to see a hint.

**Notes**:
* In this problem, you will need to constructs somewhat complicated strings.  We *strongly* encourage you to read about [Python-3 f-strings](https://www.digitalocean.com/community/tutorials/how-to-use-f-strings-to-create-strings-in-python-3) which make this process much easier than the alternatives.
* You may find `simulator.pretty_print_state` useful for debugging.
* We also highly recommend printing out the domain and problem after they have been created, and copying them into [editor.planning.domains](http://editor.planning.domains) to check whether it's possible to find a plan.
* The image in catsoop with the robot and the bears is a faithful depiction of the initial state. For example, the initial locations of the people are: "p1": (4, 0), "p2": (6, 0), "p3": (0, 6), "p4": (3, 3).
* One part of this problem that may be initially counterintuitive is the way that we'll represent locations in PDDL.
In the simulator, a location is a tuple of integers. PDDL does not support such representations -- everything needs to be just an object with a string name.
So to represent a location like (3, 5), we will make a string "l3-5" (where the first character there is a lowercase L), and we'll create an object
with that name, of type "location". We will also need a way to encode the fact that the robot can only move between adjacent locations in the grid.
In the simulator, we can compare the numeric values of locations like (3, 5) and (3, 6) to see if they are neighbors.
But in PDDL, all we have are the objects with string names, and we need to encode everything in terms of predicates.
So, we will create a predicate `(conn ?v0 - location ?v1 - location ?v2 - direction)`, which says that location `?v0` is connected to locaction `?v1`
in direction `?v2`. For example, `(conn l3-5 l3-6 right)` might appear in the initial state. We can then use these `conn` predicates in
the preconditions of a `move` operator to encode the fact that the robot can only move between adjacent locations.
* We do not recommend modelling the hospital explicitly with special objects / types / predicates. Instead, the goal should be to deliver all people to the hospital_loc, that is, `l6-6`.
In words, the goal should be "person1 is at l6-6 and person2 is at l6-6 and person3 is at l6-6 and person4 is at l6-6."
  

For reference, our solution is **121** lines of code.

In [ ]:
def find_search_and_rescue_plan():
  """Make a plan to solve the search and rescue problem in SearchAndRescueSimulator.

  When the output of this function is given to `check_sar_plan`, it should return True.

  This function should do the following:
    1. Create PDDL domain and problem strings for search and rescue.
    2. Invoke `run_planning` using the `gbf` search algorithm with the `hadd` heuristic.
    3. Convert the output of run_planning (pyperplan Operators) into actions
      that can be given to the SearchAndRescueSimulator.

  For reference, this function takes ~1-2 seconds to run with our implementation.

  Returns:
    plan: A list of actions; each action is a str, see SearchAndRescueSimulator.
  """
  # Note: using the scaffold below is optional. You are also free
  # to write your PDDL domain and problem in a different way.

  simulator = SearchAndRescueSimulator()
  init_sim_state = simulator.get_initial_state()

  # <<< TODO: fill in missing parts in the PDDL domain below >>>
  SAR_DOMAIN = """(define (domain searchandrescue)
  (:requirements :typing)
  (:types person location direction)
  
  (:constants
    down - direction
    left - direction
    right - direction
    up - direction
  )

  (:predicates
    (conn ?v0 - location ?v1 - location ?v2 - direction)
    (robot-at ?v - location)
    (person-at ?v - location ?p - person)
    (robot-free)
    (carrying ?p - person)
  )
  
  (:action move-robot
    :parameters (?from - location ?to - location ?dir - direction)
    :precondition (and
      (conn ?from ?to ?dir)
      (robot-at ?from)
    )
    :effect (and
      (not (robot-at ?from))
      (robot-at ?to)
    )
  )

  (:action pickup-person
    :parameters (?person - person ?loc - location)
    :precondition (and
      (person-at ?loc ?person)
      (robot-at ?loc)
      (robot-free)
    )
    :effect (and
      (not (robot-free))
      (carrying ?person)
      (not (person-at ?loc ?person))
    )
  )

  (:action dropoff-person
    :parameters (?person - person ?loc - location)
    :precondition (and
      (carrying ?person)
      (robot-at ?loc)
    )
    :effect (and
      (robot-free)
      (not (carrying ?person))
      (person-at ?loc ?person)
    )
  )
)"""

  # Create objects str
  objects_strs = []
  for r, c in np.argwhere(init_sim_state["obstacle_map"] == 0):
    # Creates one object for all locations in the grid that are
    # not occupied by an obstacle. For example, if r = 0, c=0, then
    # this would create an object l0-0 of type "location". Note that
    # the first character here is a lowercase L.
    objects_strs.append(f"l{r}-{c} - location")
  for p in init_sim_state["people"].keys():
    objects_strs.append(f"{p} - person")
  objects_str = " ".join(objects_strs)

  # Create init str
  deltas = {
    "up": (-1, 0),
    "down": (1, 0),
    "left": (0, -1),
    "right": (0, 1),
  }
  init_strs = []
  # Here we're going to add one (conn ...) atom for every pair
  # of clear adjacent locations. We do not have objects for
  # locations that have obstacles.
  height, width = init_sim_state["obstacle_map"].shape
  for r, c in np.argwhere(init_sim_state["obstacle_map"] == 0):
    for direction, (dr, dc) in deltas.items():
      if not (0 <= r + dr < height and 0 <= c + dc < width):
        continue
      if init_sim_state["obstacle_map"][r+dr, c+dc] == 1:
        continue
      # For example, if r == 0, c == 0, dr == 0, dc == 1, then
      # this line adds the atom (conn l0-0 l0-1 right).
      init_strs.append(f"(conn l{r}-{c} l{r+dr}-{c+dc} {direction})")
  for p in init_sim_state["people"].keys():
    #print(init_sim_state["people"][p])
    r,c = init_sim_state["people"][p]
    init_strs.append(f"(person-at l{r}-{c} {p})") # initial state of person
  robot_r, robot_c = init_sim_state["robot"]
  init_strs.append(f"(robot-at l{robot_r}-{robot_c})")
  if init_sim_state["carrying"] != None:
    per = init_sim_state["carrying"]
    init_strs.append(f"(not (robot-free))")
    init_strs.append(f"(carrying {per})")
  else:
    init_strs.append(f"(robot-free)")
  init_str = " ".join(init_strs)

  # Create goal str
  goal_strs = []
  hospital_r, hospital_c = init_sim_state["hospital_loc"]
  for p in init_sim_state["people"].keys():
    goal_strs.append(f"(person-at l{hospital_r}-{hospital_c} {p})")
  goal_str = " ".join(goal_strs)

  SAR_PROBLEM = f"""(define (problem searchandrescue) (:domain searchandrescue)
  (:objects
  {objects_str}
  )
  (:init 
  {init_str}
  )
  (:goal (and {goal_str}))
)"""

#   SAR_PROBLEM= """(define (problem searchandrescue) (:domain searchandrescue)
#   (:objects
#   l0-0 - location l0-1 - location l0-2 - location l0-3 - location l0-4 - location l0-5 - location l0-6 - location l1-0 - location l1-3 - location l1-4 - location l2-0 - location l2-1 - location l2-2 - location l2-3 - location l2-4 - location l2-5 - location l2-6 - location l3-0 - location l3-1 - location l3-3 - location l3-4 - location l3-5 - location l3-6 - location l4-0 - location l4-1 - location l4-3 - location l4-5 - location l4-6 - location l5-0 - location l5-1 - location l5-2 - location l5-3 - location l5-4 - location l5-6 - location l6-0 - location l6-2 - location l6-3 - location l6-5 - location l6-6 - location p1 - person p2 - person p3 - person p4 - person
#   )
#   (:init 
#   (conn l0-0 l1-0 down) (conn l0-0 l0-1 right) (conn l0-1 l0-0 left) (conn l0-1 l0-2 right) (conn l0-2 l0-1 left) (conn l0-2 l0-3 right) (conn l0-3 l1-3 down) (conn l0-3 l0-2 left) (conn l0-3 l0-4 right) (conn l0-4 l1-4 down) (conn l0-4 l0-3 left) (conn l0-4 l0-5 right) (conn l0-5 l0-4 left) (conn l0-5 l0-6 right) (conn l0-6 l0-5 left) (conn l1-0 l0-0 up) (conn l1-0 l2-0 down) (conn l1-3 l0-3 up) (conn l1-3 l2-3 down) (conn l1-3 l1-4 right) (conn l1-4 l0-4 up) (conn l1-4 l2-4 down) (conn l1-4 l1-3 left) (conn l2-0 l1-0 up) (conn l2-0 l3-0 down) (conn l2-0 l2-1 right) (conn l2-1 l3-1 down) (conn l2-1 l2-0 left) (conn l2-1 l2-2 right) (conn l2-2 l2-1 left) (conn l2-2 l2-3 right) (conn l2-3 l1-3 up) (conn l2-3 l3-3 down) (conn l2-3 l2-2 left) (conn l2-3 l2-4 right) (conn l2-4 l1-4 up) (conn l2-4 l3-4 down) (conn l2-4 l2-3 left) (conn l2-4 l2-5 right) (conn l2-5 l3-5 down) (conn l2-5 l2-4 left) (conn l2-5 l2-6 right) (conn l2-6 l3-6 down) (conn l2-6 l2-5 left) (conn l3-0 l2-0 up) (conn l3-0 l4-0 down) (conn l3-0 l3-1 right) (conn l3-1 l2-1 up) (conn l3-1 l4-1 down) (conn l3-1 l3-0 left) (conn l3-3 l2-3 up) (conn l3-3 l4-3 down) (conn l3-3 l3-4 right) (conn l3-4 l2-4 up) (conn l3-4 l3-3 left) (conn l3-4 l3-5 right) (conn l3-5 l2-5 up) (conn l3-5 l4-5 down) (conn l3-5 l3-4 left) (conn l3-5 l3-6 right) (conn l3-6 l2-6 up) (conn l3-6 l4-6 down) (conn l3-6 l3-5 left) (conn l4-0 l3-0 up) (conn l4-0 l5-0 down) (conn l4-0 l4-1 right) (conn l4-1 l3-1 up) (conn l4-1 l5-1 down) (conn l4-1 l4-0 left) (conn l4-3 l3-3 up) (conn l4-3 l5-3 down) (conn l4-5 l3-5 up) (conn l4-5 l4-6 right) (conn l4-6 l3-6 up) (conn l4-6 l5-6 down) (conn l4-6 l4-5 left) (conn l5-0 l4-0 up) (conn l5-0 l6-0 down) (conn l5-0 l5-1 right) (conn l5-1 l4-1 up) (conn l5-1 l5-0 left) (conn l5-1 l5-2 right) (conn l5-2 l6-2 down) (conn l5-2 l5-1 left) (conn l5-2 l5-3 right) (conn l5-3 l4-3 up) (conn l5-3 l6-3 down) (conn l5-3 l5-2 left) (conn l5-3 l5-4 right) (conn l5-4 l5-3 left) (conn l5-6 l4-6 up) (conn l5-6 l6-6 down) (conn l6-0 l5-0 up) (conn l6-2 l5-2 up) (conn l6-2 l6-3 right) (conn l6-3 l5-3 up) (conn l6-3 l6-2 left) (conn l6-5 l6-6 right) (conn l6-6 l5-6 up) (conn l6-6 l6-5 left) (person-at l4-0 p1) (person-at l6-0 p2) (person-at l0-6 p3) (person-at l3-3 p4) (robot-at l0-0) (robot-free)
#   )
#   (:goal (and (person-at l6-6 p1) (person-at l6-6 p2) (person-at l6-6 p3) (person-at l6-6 p4)))
# )"""
  
  #print(SAR_PROBLEM)
  import time
  import pandas as pd
  result = pd.DataFrame(columns= ["search","heur","time","plan length"])
  for search in ["astar", "gbf", "bfs", "sat","ehs", "ids"]:
    for h in ["blind", "hadd", "hmax", "hsa", "hff", "lmcut", "landmark"]:
      start_time = time.time()
      plan = run_planning(SAR_DOMAIN, SAR_PROBLEM, "gbf", "hadd")
      assert plan, "Failed to find a plan."
      #print(f"Planning duration: {time.time()-start_time} seconds.")
      #print(f"plan length is {len(plan)}")
      result = result.append({'search': search, "heur": h, "time": time.time()-start_time,"plan length": len(plan)}, ignore_index=True)
      print([search,h,time.time()-start_time,len(plan)])
      #print(result.head())
  result.to_csv("result.csv")
  # Convert operators to actions
  actions = []
  for op in plan:
    if "move-robot" in op.name:
      _, direction = op.name[:-1].rsplit(" ", 1)
      action = direction
    elif "pickup-person" in op.name:
      _, person, _ = op.name.split(" ")
      action = f"pickup-{person}"
    else:
      assert "dropoff-person" in op.name
      action = "dropoff"
    actions.append(action)
  print(len(actions))
  return actions

In [ ]:
SAR_DOMAIN = """(define (domain searchandrescue)
  (:requirements :typing)
  (:types person location direction)
  
  (:constants
    down - direction
    left - direction
    right - direction
    up - direction
  )

  (:predicates
    (conn ?v0 - location ?v1 - location ?v2 - direction)
    (robot-at ?v - location)
    (person-at ?v - location ?p - person)
    (robot-free)
    (carrying ?p - person)
  )
  
  (:action move-robot
    :parameters (?from - location ?to - location ?dir - direction)
    :precondition (and
      (conn ?from ?to ?dir)
      (robot-at ?from)
    )
    :effect (and
      (not (robot-at ?from))
      (robot-at ?to)
    )
  )

  (:action pickup-person
    :parameters (?person - person ?loc - location)
    :precondition (and
      (person-at ?loc ?person)
      (robot-at ?loc)
      (robot-free)
    )
    :effect (and
      (not (robot-free))
      (carrying ?person)
      (not (person-at ?loc ?person))
    )
  )

  (:action dropoff-person
    :parameters (?person - person ?loc - location)
    :precondition (and
      (carrying ?person)
      (robot-at ?loc)
    )
    :effect (and
      (robot-free)
      (not (carrying ?person))
      (person-at ?loc ?person)
    )
  )
)"""

In [ ]:
SAR_PROBLEM= """(define (problem searchandrescue) (:domain searchandrescue)
  (:objects
  l0-0 - location l0-1 - location l0-2 - location l0-3 - location l0-4 - location l0-5 - location l0-6 - location l1-0 - location l1-3 - location l1-4 - location l2-0 - location l2-1 - location l2-2 - location l2-3 - location l2-4 - location l2-5 - location l2-6 - location l3-0 - location l3-1 - location l3-3 - location l3-4 - location l3-5 - location l3-6 - location l4-0 - location l4-1 - location l4-3 - location l4-5 - location l4-6 - location l5-0 - location l5-1 - location l5-2 - location l5-3 - location l5-4 - location l5-6 - location l6-0 - location l6-2 - location l6-3 - location l6-5 - location l6-6 - location p1 - person p2 - person p3 - person p4 - person
  )
  (:init 
  (conn l0-0 l1-0 down) (conn l0-0 l0-1 right) (conn l0-1 l0-0 left) (conn l0-1 l0-2 right) (conn l0-2 l0-1 left) (conn l0-2 l0-3 right) (conn l0-3 l1-3 down) (conn l0-3 l0-2 left) (conn l0-3 l0-4 right) (conn l0-4 l1-4 down) (conn l0-4 l0-3 left) (conn l0-4 l0-5 right) (conn l0-5 l0-4 left) (conn l0-5 l0-6 right) (conn l0-6 l0-5 left) (conn l1-0 l0-0 up) (conn l1-0 l2-0 down) (conn l1-3 l0-3 up) (conn l1-3 l2-3 down) (conn l1-3 l1-4 right) (conn l1-4 l0-4 up) (conn l1-4 l2-4 down) (conn l1-4 l1-3 left) (conn l2-0 l1-0 up) (conn l2-0 l3-0 down) (conn l2-0 l2-1 right) (conn l2-1 l3-1 down) (conn l2-1 l2-0 left) (conn l2-1 l2-2 right) (conn l2-2 l2-1 left) (conn l2-2 l2-3 right) (conn l2-3 l1-3 up) (conn l2-3 l3-3 down) (conn l2-3 l2-2 left) (conn l2-3 l2-4 right) (conn l2-4 l1-4 up) (conn l2-4 l3-4 down) (conn l2-4 l2-3 left) (conn l2-4 l2-5 right) (conn l2-5 l3-5 down) (conn l2-5 l2-4 left) (conn l2-5 l2-6 right) (conn l2-6 l3-6 down) (conn l2-6 l2-5 left) (conn l3-0 l2-0 up) (conn l3-0 l4-0 down) (conn l3-0 l3-1 right) (conn l3-1 l2-1 up) (conn l3-1 l4-1 down) (conn l3-1 l3-0 left) (conn l3-3 l2-3 up) (conn l3-3 l4-3 down) (conn l3-3 l3-4 right) (conn l3-4 l2-4 up) (conn l3-4 l3-3 left) (conn l3-4 l3-5 right) (conn l3-5 l2-5 up) (conn l3-5 l4-5 down) (conn l3-5 l3-4 left) (conn l3-5 l3-6 right) (conn l3-6 l2-6 up) (conn l3-6 l4-6 down) (conn l3-6 l3-5 left) (conn l4-0 l3-0 up) (conn l4-0 l5-0 down) (conn l4-0 l4-1 right) (conn l4-1 l3-1 up) (conn l4-1 l5-1 down) (conn l4-1 l4-0 left) (conn l4-3 l3-3 up) (conn l4-3 l5-3 down) (conn l4-5 l3-5 up) (conn l4-5 l4-6 right) (conn l4-6 l3-6 up) (conn l4-6 l5-6 down) (conn l4-6 l4-5 left) (conn l5-0 l4-0 up) (conn l5-0 l6-0 down) (conn l5-0 l5-1 right) (conn l5-1 l4-1 up) (conn l5-1 l5-0 left) (conn l5-1 l5-2 right) (conn l5-2 l6-2 down) (conn l5-2 l5-1 left) (conn l5-2 l5-3 right) (conn l5-3 l4-3 up) (conn l5-3 l6-3 down) (conn l5-3 l5-2 left) (conn l5-3 l5-4 right) (conn l5-4 l5-3 left) (conn l5-6 l4-6 up) (conn l5-6 l6-6 down) (conn l6-0 l5-0 up) (conn l6-2 l5-2 up) (conn l6-2 l6-3 right) (conn l6-3 l5-3 up) (conn l6-3 l6-2 left) (conn l6-5 l6-6 right) (conn l6-6 l5-6 up) (conn l6-6 l6-5 left) (person-at l4-0 p1) (person-at l6-0 p2) (person-at l0-6 p3) (person-at l3-3 p4) (robot-at l0-0) (robot-free)
  )
  (:goal (and (person-at l6-6 p1) (person-at l6-6 p2) (person-at l6-6 p3) (person-at l6-6 p4)))
)"""

In [ ]:
import time
start_time = time.time()
plan = run_planning(SAR_DOMAIN, SAR_PROBLEM, "sat", "blind")
assert plan, "Failed to find a plan."
    #print(f"Planning duration: {time.time()-start_time} seconds.")
    #print(f"plan length is {len(plan)}")
print([time.time()-start_time,len(plan)])
    #print(result.head())


TypeError: ignored

In [ ]:
lis = []
for search in ["astar", "gbf", "bfs", "sat"]:
  for h in [ "hadd", "hmax", "hsa", "hff", "lmcut", "landmark"]:
    lis.append((search,h))
print(lis)

[('astar', 'hadd'), ('astar', 'hmax'), ('astar', 'hsa'), ('astar', 'hff'), ('astar', 'lmcut'), ('astar', 'landmark'), ('gbf', 'hadd'), ('gbf', 'hmax'), ('gbf', 'hsa'), ('gbf', 'hff'), ('gbf', 'lmcut'), ('gbf', 'landmark'), ('bfs', 'hadd'), ('bfs', 'hmax'), ('bfs', 'hsa'), ('bfs', 'hff'), ('bfs', 'lmcut'), ('bfs', 'landmark'), ('sat', 'hadd'), ('sat', 'hmax'), ('sat', 'hsa'), ('sat', 'hff'), ('sat', 'lmcut'), ('sat', 'landmark')]


In [ ]:
from google.colab import files
files.download("result.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Tests

In [ ]:
def sar_test1():
  assert check_sar_plan(find_search_and_rescue_plan())

sar_test1()
print('Tests passed.')

['astar', 'blind', 2.147775411605835, 88]
['astar', 'hadd', 2.1365551948547363, 88]
['astar', 'hmax', 2.102835178375244, 88]
['astar', 'hsa', 2.1224024295806885, 88]
['astar', 'hff', 2.1304664611816406, 88]
['astar', 'lmcut', 2.1171610355377197, 88]
['astar', 'landmark', 2.1003196239471436, 88]
['gbf', 'blind', 2.1005465984344482, 88]
['gbf', 'hadd', 2.1310391426086426, 88]
['gbf', 'hmax', 2.1196277141571045, 88]
['gbf', 'hsa', 2.0869712829589844, 88]
['gbf', 'hff', 2.1733527183532715, 88]
['gbf', 'lmcut', 2.0929372310638428, 88]
['gbf', 'landmark', 2.1029551029205322, 88]
['bfs', 'blind', 2.0953335762023926, 88]
['bfs', 'hadd', 2.1102709770202637, 88]
['bfs', 'hmax', 2.107492208480835, 88]
['bfs', 'hsa', 2.1467573642730713, 88]
['bfs', 'hff', 2.1084141731262207, 88]
['bfs', 'lmcut', 2.1003658771514893, 88]
['bfs', 'landmark', 2.180860757827759, 88]
['sat', 'blind', 2.1126246452331543, 88]
['sat', 'hadd', 2.114438056945801, 88]
['sat', 'hmax', 2.1431081295013428, 88]
['sat', 'hsa', 2.0

### Search algorithm & heuristic comparisons
Let's now compare different search algorithms and
heuristics on the search and rescue problem above.

The search algs available in pyperplan are: `astar, wastar, gbf, bfs, ehs, ids, sat`.

The heuristics available in pyperplan are: `blind, hadd, hmax, hsa, hff, lmcut, landmark`.

Unfortunately the documentation for pyperplan is limited at the moment, but if you
are curious to learn more about its internals, the code is open-sourced here:

  https://github.com/aibasel/pyperplan

Choose 8 combinations of (search algorithm, heuristic). For each, record the planning
duration in seconds for Search and Rescue. If planning takes more than 30 seconds,
you can kill the process and record "timeout".  Also record the length of the plans.

<div class="question question-multiplechoice">
<b>Submission Material:</b> In your submitted pdf, please include a table with headers
"Search Algorithm", "Heuristic", "Duration (s)", and "Plan Length" with 8 rows.
</div>
